In [1]:
!CXXFLAGS+=-stdlib=libc++ maturin develop --release

🔗 Found pyo3 bindings
🐍 Found CPython 3.11 at /Users/josh/josh/projects/tech/flow/.venv/bin/python
💻 Using `MACOSX_DEPLOYMENT_TARGET=10.7` for x86_64-apple-darwin by default
   Compiling flow v0.1.0 (/Users/josh/josh/projects/tech/flow)
    Finished release [optimized + debuginfo] target(s) in 12.45s              
📦 Built wheel for CPython 3.11 to /var/folders/w9/c8qym84j2pg4fjvx_bzr84bc0000gn/T/.tmpMUUqi5/flow-0.1.0-cp311-cp311-macosx_10_7_x86_64.whl
🛠 Installed flow-0.1.0


In [3]:
import contextplot
import matplotlib
import matplotlib.collections
import matplotlib.patches
import numpy as np
import tqdm

import flow

contextplot.set_defaults()

US_PER_S = 1e6

example = 'bump_1d'
# example = 'ramp_1d'

num_cells = 60
solver = getattr(flow, example)(num_cells, 20)
x_vertices = solver.grid.x_axis.vertices
x_centers = solver.grid.x_axis.centers

dt_max_us = int(0.01 * US_PER_S)
frame_rate_hz = 30
t_max_us = int(5 * US_PER_S)
speedup_factor = 0.35

courant_margin = 10
dt_plot_us = int(1 / frame_rate_hz * speedup_factor * US_PER_S)

def frame_index(t_us: int):
    return t_us // dt_plot_us

with contextplot.context_video(f'videos/{example}.mp4', 1, size_inches=(6.5, 3), dpi=200, frame_rate_hz=frame_rate_hz) as cv:
    try:
        t_us = 0
        bar = tqdm.tqdm(total=t_max_us)
        should_plot = True
        while t_us < t_max_us:
            if should_plot:
                with cv.next_frame() as cp:
                    z_lattice = solver.z_lattice
                    pressure = solver.pressure
                    velocity = solver.velocity

                    poly_verts = []
                    pressures = []
                    for i in range(z_lattice.shape[0] - 1):
                        for j in range(z_lattice.shape[2] - 1):
                            poly_verts.append([
                                [x_vertices[i], z_lattice[i, 0, j]],
                                [x_vertices[i + 1], z_lattice[i + 1, 0, j]],
                                [x_vertices[i + 1], z_lattice[i + 1, 0, j + 1]],
                                [x_vertices[i], z_lattice[i, 0, j + 1]],
                            ])
                            pressures.append(pressure[i, 0, j, 1])
                    poly_collection = matplotlib.collections.PolyCollection(
                        poly_verts,
                        closed=True,
                        array=pressures,
                        cmap=matplotlib.cm.viridis,
                        edgecolors='black',
                        linewidth=0.2,
                        snap=True,
                        norm=matplotlib.colors.Normalize(vmin=0, vmax=10),
                    )
                    outs = cp.ax_raw.add_collection(poly_collection)
                    _ = cp.figure.colorbar(outs, label='Kinematic pressure')

                    cp.ax.quiver(
                        np.broadcast_to(x_centers[:, np.newaxis], z_lattice[:-1, 0, :-1].shape),
                        0.25 * (z_lattice[:-1, 0, :-1] + z_lattice[1:, 0, :-1] + z_lattice[:-1, 0, 1:] + z_lattice[1:, 0, 1:]),
                        velocity[:, 0, :, 0, 0],
                        velocity[:, 0, :, 0, 2],
                        pivot='tail',
                        scale=5,
                        scale_units='inches',
                        alpha=0.6,
                    )

                    cp.ax.set(
                        xlim=[solver.grid.x_axis.vertices[0], solver.grid.x_axis.vertices[-1]],
                        ylim=[0, 1.5],
                        aspect='equal',
                        title=f'{example}, t = {t_us / US_PER_S:.3f} s',
                    )

            dt_us = min(dt_max_us, int(solver.courant_dt / courant_margin * US_PER_S))

            current_frame_index = frame_index(t_us=t_us)
            future_frame_index = frame_index(t_us=t_us + dt_us)
            if current_frame_index != future_frame_index:
                dt_us = (current_frame_index + 1) * dt_plot_us - t_us
                should_plot = True
            else:
                should_plot = False
            
            bar.set_description(f'dt: {dt_us / US_PER_S:.6f}')

            solver.step(dt_us / US_PER_S)
            t_us += dt_us
            # print(t_us, dt_us)
            if t_us > t_max_us:
                break
            _ = bar.update(dt_us)
    except KeyboardInterrupt:
        pass
    except BaseException:
        pass
    finally:
        bar.close()

  0%|          | 0/5000000 [00:00<?, ?it/s]

dt: 0.000000:  30%|██▉       | 1487936/5000000 [01:05<02:35, 22643.86it/s]stack backtrace:
   0: rust_begin_unwind
             at /rustc/90c541806f23a127002de5b4038be731ba1458ca/library/std/src/panicking.rs:578:5
   1: core::panicking::panic_fmt
             at /rustc/90c541806f23a127002de5b4038be731ba1458ca/library/core/src/panicking.rs:67:14
   2: <i64 as klu_rs::raw::KluIndex>::check_status
             at /Users/josh/.cargo/registry/src/index.crates.io-6f17d22bba15001f/klu-rs-0.4.0/src/raw.rs:810:18
   3: klu_rs::KluSettings<I>::check_status
             at /Users/josh/.cargo/registry/src/index.crates.io-6f17d22bba15001f/klu-rs-0.4.0/src/lib.rs:36:9
   4: klu_rs::FixedKluMatrix<I,D>::lu_factorize
             at /Users/josh/.cargo/registry/src/index.crates.io-6f17d22bba15001f/klu-rs-0.4.0/src/lib.rs:206:9
   5: flow::linalg::solve_linear_sytem_klu
             at ./src/linalg.rs:69:14
   6: flow::linalg::LinearSolver::solve
             at ./src/linalg.rs:21:29
   7: flow::implici